# Constructing Agents

## An agent is a thing inside your environnment

The initial **Thing** class is really just a shell. We'll want to define whether the thing is *alive*, how to display our thing's state, and how to display maybe a picture of our thing, like if our thing were a function we could make a picture. The **Agent** class is a subclass that performs actions based on what it perceives in the environment. To keep things general, this agent class will take in a user-defined FUNCTION that turns perceptions into actions.

## The Environment class
Is a shell for all environments. It owns **things** and **agents**. It specifies:
* The thing classes it can hold. These things can be just things (like dirt) or agents (like vacuums that can do stuff)
* What it can perceive (percept classes -- like what sensors are on my robot?)
* What it can do. Like if a vacuum sucks up dirt, then it can change the amount of dirt in its environment.
* Specify a default location for new things, like where more dirt might go.
* Specify changes we won't allow ("exogenous_change")
* Tell us if all of the agents are dead
* Perform one time step in our environmental "game" definition.
  * Each agent gets to perceive its state
  * Each agent gets to perform an action
* Perform a bunch of steps
* List all the things at a location
* List some things at a location?
* Add a thing at a location (or default location)
* Delete a specified thing

## The Direction Class
* Specify a heading (**R**ight, **L**eft, **U**p, **Do**wn)
* Move Forward

## Environments on a plane XYEnvironment(Environment)
* Rectangle with width and height
  * also initialize a list of observers -- this might be a list provided to the GUI that tells us when things change
* Things near a location (based on perceptible_distance = 1 or specified radius)
* Return what things I can see
* Execute an action
  * bump against an edge
  * turn left or right
  * move forward
  * grab a thing
  * release a thing
* Add observers who get to find out what's happened
* Move to where I say to move:
  * If there's an *Obstacle* at my destination I bump. Obstacles are their own trivial class that can be extended into more 
    complicated obstacles that are sets of coordinates.
  * Otherwise tell all observers the thing moved and remove the thing from the old destination and put it in the new one
  * Return True/False for whether or not I moved the thing
* Add a thing to a location. Say what to do if there's a thing there.
* Check to see if the location some jerk specified is actually in my rectangle.
* Randomly choose a location in my rectangle, and maybe I'll list some patches that aren't allowed.
* Delete a thing from the environment. If that thing is an agent drop everything it's holding.
* Add walls so the vacuum doesn't fall down the stairs. A *Wall* is its own trivial class.
* Describe the new heading after a turn happens

## GraphicEnvironment(XYEnvironment)
Handles the GUI

# Finally let's make a vacuum environment
<p align="center>
<img src="images/vacuum.svg">
</p>

* Initialize Dirt as a Trivial thing

* Extend the XYEnvironment to **VacuumEnvironment**:
  * The things are Wall, Dirt, and Four agents (reflux, random, tabledriven, and modelbased) for vacuum behavior. We'll get to those.
  * The environment knows if an agent (a.k.a. a vacuum) is  standing in dirt and if it will bump into something if it moves forward.
  * An agent can execute an action:
    * If the action is suck it gets 100 points (**performance**) and it deletes the dirt, otherwise the performance is -1 and the action is exected according to the agent's logic.

The only fully-defined environment is for two grids:

### The ReflexVacuumAgent

This is actually only for the trivial world with 2 grids.

```python
class TrivialVacuumEnvironment(Environment):
    """This environment has two locations, A and B. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status. This serves as an example of how to implement a simple
    Environment."""

    def __init__(self):
        super().__init__()
        self.status = {loc_A: random.choice(['Clean', 'Dirty']),
                       loc_B: random.choice(['Clean', 'Dirty'])}

    def thing_classes(self):
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent, TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        return agent.location, self.status[agent.location]

    def execute_action(self, agent, action):
        """Change agent's location and/or location's status; track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        if action == 'Right':
            agent.location = loc_B
            agent.performance -= 1
        elif action == 'Left':
            agent.location = loc_A
            agent.performance -= 1
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                agent.performance += 10
            self.status[agent.location] = 'Clean'

    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([loc_A, loc_B])
```

<p align="center">
<img src="images/simple_reflex_agent.jpg">
</p>

Perceive the location and status
* If the status is dirty, suck
* Otherwise move to the other location

But more generally it is
```python
def SimpleReflexAgentProgram(rules, interpret_input):
    """
    [Figure 2.10]
    This agent takes action based solely on the percept.
    """

    def program(percept):
        state = interpret_input(percept)
        rule = rule_match(state, rules)
        action = rule.action
        return action

    return program

```

So maybe we'd say
* if the location is dirty, suck (performance +100)
* Otherwise if no bump move forward (performance -1)
* Otherwise move heading to the left or right -- probably want to be random (performance -1)

state = (location, heading, status)
Note that the examples don't have a heading as part of the state, but it would be necessary for the environment to track to know what happens when we say to move right or left. (We don't need to know to determine action, though. If there's a bump we need to change the heading.)


In [10]:
from agents import ReflexVacuumAgent, TrivialVacuumEnvironment

In [12]:
agent = ReflexVacuumAgent()
environment = TrivialVacuumEnvironment()
environment.add_thing(agent)
environment.status


{(0, 0): 'Dirty', (1, 0): 'Clean'}

In [13]:
environment.run()
environment.status

{(0, 0): 'Clean', (1, 0): 'Clean'}

In [6]:
from agents import Agent

The TraceAgent function reports on the agents perception-action movements.

In [ ]:
from agents import TraceAgent